<!-- dom:TITLE: Programmation Python  pour les mathématiques -->
# Python Programming for Mathematics
<!-- dom:AUTHOR: Julien Guillod at [Sorbonne Université](http://www.sorbonne-universite.fr/), -->
<!-- Author: -->  
**Julien Guillod**, [Sorbonne Université](http://www.sorbonne-universite.fr/),
Licensed <a href="https://creativecommons.org/licenses/by-nc-nd/4.0/">CC BY-NC-ND</a>


All chapters are available in
[HTML](https://python.guillod.org//) and [PDF](https://python.guillod.org//python.pdf).
This notebook is also executed on [mybinder](https://mybinder.org/v2/gh/juguillod/python/master?filepath=chap11.ipynb).






# 11 Cryptography
<div id="ch:cryptographie"></div>

Since the Caesar's cipher, different cryptographic methods allowing human to transmit secret messages have evolved in parallel with the progresses and efforts to crack them. We will study the Vigenere cipher which is an improvision of the Caesar's cipher, then we will see the method to crack this cipher. And then, we are introduced to the RSA cryptosystem, one of the most used asymmetric cryptographic methods today.

**This chapter covers:**

* Vigenere cipher

* Greatest common divisor

* Importing text

* Primes and pseudo-primes

* Fermat's little problem

* Euclide's algorithm

* Rabin-Miller's algorithm

* Optimization by Python's decorator

* RSA asymmetric cryptography





<!-- --- begin exercise --- -->

# Exercise 11.1: Vigenere cipher

The Vigenere cipher consists of choosing a secret keyword (upper-cased) and then transform it into a vector in which each element is the position of its corresponding letter in the alphabet. For example, `ASECRET` corresponds to (0, 18, 4, 2, 17, 4, 19). To encrypt a text (upper-cased without space and punctuation) with the keyword `ASECRET`, one needs to shift the first letter in the text by the offset of 0, the second by 18, the third by 4, and so on, while having the keyword looping forever. Details are available on the [Wikipedia page](https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher).


**a)**
Write a function `to_int(s)` transforming a character to its character in the alphabet and the inverse function `to_chr(i)`.

<!-- --- begin hint in exercise --- -->

**Hints:**
Use Python's built-in functions `ord` and `chr`.

<!-- --- end hint in exercise --- -->



In [1]:
def to_int(s):
    return ord(s)-65

def to_chr(i):
    return chr(i+65)

print(to_int('S'))
print(to_chr(18))

18
S



**b)**
Write a function `encrypt(text, key)` encrypting `text` with the keyword `key`.

In [2]:
from itertools import cycle

def encrypt(text, key):
    code = [str((to_int(char_text) + to_int(char_key))%26) for char_text, char_key in zip(text, cycle(key))]
    return " ".join(code)

text = 'VIVELAFRANCE'
key = 'TRUNG'
encrypt(text, key)

'14 25 15 17 17 19 22 11 13 19 21 21'


**c)**
Write a function decrypting a text knowing the keyword.

In [3]:
def decrypt(code, key):
    code_int = list(map(int, code.split(" ")))
    text = [to_chr((num - to_int(char))%26) for num, char in zip(code_int, cycle(key))]
    return "".join(text)

code = '14 25 15 17 17 19 22 11 13 19 21 21'
decrypt(code, key)

'VIVELAFRANCE'



<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercise 11.2: <span style="color:red">!</span> Cracking the Vigenere cipher

Charles Babbage was the first to crack the Vigenere cipher. The idea is that 3 consecutive letters appearing multiple times in the encrypted text are likely to be the same letters in the text encrypted by the same letters in the keyword. This is even more probable with a group of 4 letters. Thus the offset between two same groups of encrypted letters is a multiplier of the keyword's length. For example, if the repetition of a group is offset by 28 letters, then another is 91, the greatest common divisor of 28 and 91 is 7, hence it is probable that the keyword has 7 letters. Then, knowing the length of the keyword, one can proceed based on the fact that the letter "E" is the most common letter in French (and in most western languages). In order to succeed with this strategy, the text must be long enough.


**a)**
Write a function calculating the greatest common divisor (GCD) between two numbers. Write another function calculating the GCD of a list of numbers.


In [4]:
from functools import reduce

def gcd(a, b):
    while b != 0:
        r = a % b
        a, b = b, r
    return a

def gcd_list(nums):
    return reduce(gcd, nums)

gcd_list([28,91,21,49,77])

7


**b)**
Visit the site of the [Gutenberg project](http://www.gutenberg.org/browse/languages/fr), choose a French text that you prefer then download it under plain text format. Write a function converting the text to uppercase and removing all punctuations and special characters.

<!-- --- begin hint in exercise --- -->

**Hints:**
To convert the text to the desired format, it is possible to use the following function:

In [5]:
import unicodedata, re

def convert_upper(text):
    # Convert letters to uppercase
    text = text.upper()
    # Un-accent letters
    text = unicodedata.normalize('NFKD', text)
    # Remove special characters
    regex = re.compile('[^a-zA-Z]')
    text = regex.sub('', text)
    return text

original_text = "C'est une matière grave à traiter dans les annales d'un pays comme la France, que l'Histoire des salons de Paris. Depuis une certaine époque, cette histoire se trouve étroitement liée à celle du pays, et surtout aux intrigues toujours attachées aux plans politiques qui si longtemps bouleversèrent le royaume. L'époque de la naissance de la société en France, dans l'acception positive de ce mot, remonte au règne du cardinal de Richelieu. En rappelant la noblesse autour du trône, en lui assignant des fonctions, créant pour elle des charges et des places, dont son orgueil devait jouir, Richelieu donna de la sécurité à la Couronne, sans cesse exposée par les caprices d'un grand seigneur, comme le duc de Bouillon, le duc de Longueville, le duc de Montbazon, et une foule d'autres qui, plus libres dans leurs châteaux, étaient conspirateurs par état et par goût. La réunion de tous ces grands noms autour du trône lui donna plus que de la sécurité, il en doubla la majesté; mais aussi le premier coup fut porté à la noblesse: elle n'eut plus dès-lors de ces grandes entreprises à conduire, qui mettaient en péril à la fois la tête des conspirateurs et le sort de l'État. Richelieu, avec cette justesse de coup d'œil qui lui fit voir le mal sous toutes ses faces, le conjura en appelant la noblesse au Louvre; mais il ne put l'empêcher de conserver ce qui était inhérent à sa nature, toujours portée à l'intrigue et au mouvement. C'est ainsi que, même sous le ministère de Richelieu, on conspirait dans Paris chez les femmes de haute importance, telles que la princesse Palatine, madame de Chevreuse, madame de Longueville, et une foule de femmes toutes-puissantes par leur position dans le monde, leur esprit ou leur beauté... Avides de pouvoir, ces mêmes femmes saisirent, aussitôt qu'elles le comprirent, le moyen que le cardinal lui-même leur avait laissé. Elles régnaient avant dans une ville éloignée, un château-fort habité par des hommes dont le meilleur et le plus agréable n'était souvent qu'un mal-appris; maintenant elles étaient au milieu de Paris, de ce lieu qui, même à cette époque, où il n'était pas embelli par tout le prestige de la Société Parisienne, de cette société qui si longtemps donna partout, en Europe, le modèle du goût et des façons parfaitement nobles et élégantes, formait déjà le parfait gentilhomme. Ce fut alors dans chaque maison particulière qu'il fallut chercher une reine donnant ses lois et dirigeant une opinion. C'est dans les Mémoires du cardinal de Retz, dans ce livre-modèle, qu'on peut reconnaître cette vérité, dans ceux de madame de Motteville. Voyez l'abbé de Gondy lui-même arrivant chez madame de Chevreuse. Suivez-le dans les détours qu'on lui fait parcourir une nuit, pour parvenir jusqu'à la duchesse, lorsqu'il est cependant l'ami de sa fille[1]. Vous le rencontrez ensuite dans les salons à peine organisés, avec M. de Beaufort, M. le duc de Nemours, M. de La Rochefoucauld, et vous êtes admis aux secrets importants de l'époque.... Le salon de madame de Longueville, celui de Mademoiselle, de madame de Lafayette, deviennent comme des clubs à une époque révolutionnaire. Gaston, mannequin de l'abbé de Larivière, dirige tout du Palais-Royal, et la Cour elle-même n'est plus qu'un instrument."

text = convert_upper(original_text)
text

'CESTUNEMATIEREGRAVEATRAITERDANSLESANNALESDUNPAYSCOMMELAFRANCEQUELHISTOIREDESSALONSDEPARISDEPUISUNECERTAINEEPOQUECETTEHISTOIRESETROUVEETROITEMENTLIEEACELLEDUPAYSETSURTOUTAUXINTRIGUESTOUJOURSATTACHEESAUXPLANSPOLITIQUESQUISILONGTEMPSBOULEVERSERENTLEROYAUMELEPOQUEDELANAISSANCEDELASOCIETEENFRANCEDANSLACCEPTIONPOSITIVEDECEMOTREMONTEAUREGNEDUCARDINALDERICHELIEUENRAPPELANTLANOBLESSEAUTOURDUTRONEENLUIASSIGNANTDESFONCTIONSCREANTPOURELLEDESCHARGESETDESPLACESDONTSONORGUEILDEVAITJOUIRRICHELIEUDONNADELASECURITEALACOURONNESANSCESSEEXPOSEEPARLESCAPRICESDUNGRANDSEIGNEURCOMMELEDUCDEBOUILLONLEDUCDELONGUEVILLELEDUCDEMONTBAZONETUNEFOULEDAUTRESQUIPLUSLIBRESDANSLEURSCHATEAUXETAIENTCONSPIRATEURSPARETATETPARGOUTLAREUNIONDETOUSCESGRANDSNOMSAUTOURDUTRONELUIDONNAPLUSQUEDELASECURITEILENDOUBLALAMAJESTEMAISAUSSILEPREMIERCOUPFUTPORTEALANOBLESSEELLENEUTPLUSDESLORSDECESGRANDESENTREPRISESACONDUIREQUIMETTAIENTENPERILALAFOISLATETEDESCONSPIRATEURSETLESORTDELETATRICHELIEUAVECCETTEJUSTESSEDECOUPDILQUILUIFITVOIRLEMALSOUSTOUT


**c)**
Encrypt about 1000 characters in the middle of the text with a keyword of your choice. Then write a function determining the length of the keyword using the mentionned method by looking at groups of letters.

<!-- --- begin hint in exercise --- -->

**Hints:**
First create a dictionary in which the keys are groups of tree letters and the values are positions of these groups in the text. Then create a list of distance between these occurrences, calculate the GCD of these distances. If GCD equals 1 or too small, retry with groups of one letter longer.

<!-- --- end hint in exercise --- -->



In [6]:
def n_letters_distances(crypted_msg, n):
    dic = dict()
    
    for i in range(len(crypted_msg) - n):
        three_letters = crypted_msg[i: i+n]
        if three_letters not in dic:
            dic[three_letters] = []
        dic[three_letters].append(i)
    
    set_distance = set()
    for key in dic.keys():
        if len(dic[key]) == 1:
            continue
        distance_to_1st = dic[key][0]
        for num in dic[key]:
            if num - distance_to_1st != 0:
                set_distance.add(num - distance_to_1st)
    return set_distance

def key_length(code):
    crypted_msg = "".join(map(lambda x: to_chr(int(x)), code.split(" ")))
    n = 3
    while gcd_list(n_letters_distances(crypted_msg, n)) <= 3:  # We assume that all keys must be complicated enough to have at least 4 characters
        n += 1
        if n == 10:
            return ("Cannot determine key length")
    return gcd_list(n_letters_distances(crypted_msg, n))

keys = ["PARIS", "FRANCE", "ANTIDISESTABLISHMENTARIANISM"]
for key in keys:
    code = encrypt(text, key)
    print(f"Key: {key}, real length: {len(key)}, predicted length: {key_length(code)}")

Key: PARIS, real length: 5, predicted length: 5
Key: FRANCE, real length: 6, predicted length: 6
Key: ANTIDISESTABLISHMENTARIANISM, real length: 28, predicted length: 28



**d)**
Write a function decrypting an encrypted message by returning the keyword.

<!-- --- begin hint in exercise --- -->

**Hints:**
To determine the first letter of the keyword, we must calculate the number of occurrences of the 26 letters in the encrypted message which are encrypted by this first letter of the keyword. The letter that occurs the most should correspond to the letter **"E"**. We can repeat this step to determine the rest of the keyword.

<!-- --- end hint in exercise --- -->



In [7]:
def guess_key(code):
    length = key_length(code)
    letters_list = code.split(" ")
    
    cryptograms = [[]]*length
    for i, letter in enumerate(letters_list):
        if not cryptograms[i % length]:
            cryptograms[i % length] = [0]*26
        cryptograms[i % length][int(letter)] += 1
    key = ""
    for crypto in cryptograms:
        max_where = max(range(len(crypto)), key=lambda i: crypto[i])
        key_char = to_chr((max_where - 4)%26)
        key += key_char
    return key

real_keys = ["PARIS", "FRANCE", "VIVELAFRANCE", "DAMQUANGTRUNG", "ANTIDISESTABLISHMENTARIANISM"]
for key in real_keys:
    code = encrypt(text, key)
    print(f"Real key: {key}. Guessed key: {guess_key(code)}")

Real key: PARIS. Guessed key: PARIS
Real key: FRANCE. Guessed key: FRANCE
Real key: VIVELAFRANCE. Guessed key: VIVELAFRANCE
Real key: DAMQUANGTRUNG. Guessed key: DAMQUANGTRUNG
Real key: ANTIDISESTABLISHMENTARIANISM. Guessed key: ANTIDISESTABLISHWANTARIAXISM


> ### __The moral of the story is, the shorter and more common the key, the easier it is to crack using basic methods__


<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercise 11.3: Generate prime numbers

Most of cryptographic algorithms actually are based on the use of very big primes. The goal for this exercise is to write a function generating, in a relatively fast execution time, the so-called *pseudo-prime* numbers, meaning extremely probably prime. The first step is to generate a big random number using bits, or its binary form. Then we perform a primality test to decide if this number is prime or not. Denoting $\pi(n)$  the number of primes less than or equal to $n$ then, asymtotically $\pi(n) \approx \frac{n}{\ln n}$. Thus, if we choose randomly a number less than $n$ the probability of it being prime is about $1/\ln(n)$. For example, to generate a prime of 1024 bits (the minimum value today for a strengthened security), meaning a number less than $2^{1024}$, we have to try $\ln(2^{1024}) \approx 710$ random numbers before finding one that is prime. Since even numbers are clearly non-prime, we should only have to test in average $355$ numbers.


**a)**
Write a function generating a random odd number of $k$ bits, meaning its value between $2^{k-1}$ and $2^{k}$.

<!-- --- begin hint in exercise --- -->

**Hints:**
Bitwise operations can be useful. Documentation is available [here](https://docs.python.org/fr/3.6/reference/expressions.html#binary-bitwise-operations).

<!-- --- end hint in exercise --- -->



In [8]:
import random

def random_num(k):
    num = "1"  # Must start with 1 in order to ensure that num is k-bit
    for _ in range(k-2):
        num += random.choice(['0', '1'])
    num += "1"  # Must end with 1 to ensure its oddity
    return int(num, 2)

random_num(100)

1095522523804860127851777581135

The most straightforward way to determine if a number $n$ is prime is to try dividing it by all numbers between $1 < d < n$. Two things to remember: do not test with any even numbers other than 2, and stop the test when $d$ reaches beyond $\sqrt{n}$.

**b)**
Write an algorithm `isprime(n)` determining if a number is prime or not.


In [9]:
from math import sqrt, ceil

def isprime(n):
    """ Trivial method by trial division with all odd numbers from 3 up to sqrt(n) """
    if n % 2 == 0:
        return False
    elif n == 3:
        return True
    for d in range(3, ceil(sqrt(n))+1 , 2):
        if isprime(d) and n % d == 0:
            return False
    return True

isprime(99971), isprime(121)

(True, False)


**c)**
Write a function `generate(k, primality)` generating a random prime of `k` bits by testing its primality with the `primality` test. Test with `primality=isprime`. Is it reasonable to even hope for a prime of 1024 bits with this algorithm?


In [12]:
import time

def generate(k, primality, N=10000):
    for i in range(N):  # Only try N times, if all fail then return
        num = random_num(k)
        if primality(num):
            return num
    return f"Failed to generate a {k}-bit prime number"

try:
    start = time.time()
    generate(100, isprime)
except KeyboardInterrupt:
    end = time.time()
    print("Interupted kernel after", end - start, "minutes because this is taking way to looooong.")

Interupted kernel after 27.0 minutes because this is taking way to looooong.


> #### There is really no hope for doing this the trivial way with a random odd number as ___small___ as $2^{100}$, let alone $2^{1024}$


<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercise 11.4: Generate pseudo-primes

The above algorithm generating primes being unusable to return a big prime, another probabilistic approac, should be considered. A probabilistic test of primality decides that a number is prime if it is ***very likely*** to be prime. Such number is called pseudo-prime. However a probabilistic test can also be faulty where it mistakenly determines a non-prime to be one.

The easiest primality test is based on Fermat's little theorem: if $n$ is prime, then $a^{n-1}=1 \pmod n$ for $1 \leq a \leq n-1$. If we find a value of $a$ so that $a^{n-1}\neq1 \pmod n$ then $n$ is not prime. Fermat's primality test tries $N$ values of $a$ chosen randomly and if $a^{n-1}=1 \pmod n$ for all these $N$ values then it will decide that $n$ is probably prime. The Carmichael's numbers are not prime but satisfy anyway $a^{n-1}=1 \pmod n$ for $a$ co-prime with $n$. Some of the first Carmichael's numbers include 561, 1105 and 1729. If $n$ isn't Carmichael's then the probability of a faulty Fermat's test is $2^{-N}$. By choosing for example $N=128$, we obtain a probability of a faulty test below $3\times 10^{-39}$.

**a)**
Write a function implementing the Fermat's primality test. Use this test to generate random primes.

<!-- --- begin hint in exercise --- -->

**Hints:**
Look at the documentation of the function `pow` for a quick implementation. If OpenSSL is installed, it is then easy to verify if a number is prime with the command `!openssl prime 11` for example for 11.

<!-- --- end hint in exercise --- -->



In [13]:
# The set of Carmichael numbers below is extended to more values.
# Credits: http://oeis.org/wiki/Carmichael_numbers

carmichael = {561, 1105, 1729, 2465, 2821, 6601, 8911, 10585, 15841, 29341, 46657, 52633,
              115921, 162401, 252601, 294409, 314821, 334153, 399001, 410041, 488881, 512461,
              530881, 1024651, 1152271, 1193221, 41041, 62745, 63973, 75361, 101101, 126217,
              172081, 188461, 278545, 340561, 449065, 552721, 656601, 658801, 670033, 748657,
              825265, 838201, 852841, 997633, 1033669, 1050985, 1082809, 1569457}

def isprime_fermat(n, N=128, lower_bound=2):
    if n in {2, 3}:
        return False
    for _ in range(N):
        a = random.randint(lower_bound, n-1)
        while a in carmichael or a%2 == 0:
            a = random.randint(2, n-1)
        if pow(a, n-1, n) != 1:
            return False
    return True

In [14]:
num = generate(1024, isprime_fermat)
num

144777681876057837847864663383089832722480177260837457056179680076703851991169291363496018669247308500501498586015674960443350726094109941425679032314163948049762341176991120669360872551293330582663061292997854235780281953891265857146085575195999677540784146956955463715785529860862933402671027869023277965003

In [15]:
!openssl prime {num}

CE2B96539D1733A0EB7B79343F228128B4DD4C7B435F671B3F636BF20A39B91686265BCF63B32418F0DDD07F5B8469A33DB41BE58051CF468C131DD7F70965930CA8189BEB9A96A6C40E5E06DBF574ED94505284C6D223917E2B4AC82DF1F814FBD8C2A92C1DB339283F93E4F08EDA86C91FD2388F110A08E567619FEC145ECB (144777681876057837847864663383089832722480177260837457056179680076703851991169291363496018669247308500501498586015674960443350726094109941425679032314163948049762341176991120669360872551293330582663061292997854235780281953891265857146085575195999677540784146956955463715785529860862933402671027869023277965003) is prime


In [16]:
%%timeit
generate(1024, isprime_fermat)

The slowest run took 6.84 times longer than the fastest. This could mean that an intermediate result is being cached.
2.87 s ± 1.94 s per loop (mean ± std. dev. of 7 runs, 1 loop each)



**b)**
<span style="color:red">!</span> Improve the speed of the above algorithm by testing first if $n$ is divisible by primes less than 1000 before applying the Fermat's method.

In [17]:
import numpy as np

def primes_lte(n):
    # To quickly generate an array of primes less than or equal to n using Numpy, we first initiate a boolean array of size n+1
    # representing numbers from 0 to n, with first only True as values. We know 0 and 1 aren't prime, we then start from 2. During
    # the iteration through this bool array, if we hit a True at position i, we then make every other i-th values False, indicating
    # that these are multiplier of i and should not be prime. This algorithm should be very fast using Numpy's indexing technique.
    # When the iteration is finished, we then use the bool array as a mask for the range from 0 to n+1 which then returns the
    # list of primes less than or equal to n.
    
    isprime = np.ones(n+1, dtype=bool)
    isprime[0:2] = False
    for i in range (2, n+1):
        if isprime[i]:
            isprime[2*i::i] = False
    return np.arange(n+1)[isprime]

def isprime_fermat_faster(n, N=128, k=1000):
    first_few_primes = primes_lte(k)
    for prime in first_few_primes:
        if n % prime == 0:
            return False
        
    maxi = first_few_primes[-1]
    return isprime_fermat(n, N, lower_bound=maxi)

In [18]:
%%timeit
generate(1024, isprime_fermat_faster)

872 ms ± 430 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Fermat's primality test generates big pseudo-primes with a good probability to be effectively prime. The main problem comes from the existence of Carmichael's numbers which are excluded from this probability. The Miller-Rabin's test can avoid this problem.

**c)**
<span style="color:red">!!</span> Understand and implement the algorithm on the [Wikipedia page](https://fr.wikipedia.org/wiki/Test_de_primalité_de_Miller-Rabin).



In [19]:
def isprime_miller_rabin(n, k=10):
    if n in {2, 3, 5, 7, 11}:
        return True
    s, d = 0, n-1
    while d % 2 == 0:
        s += 1
        d >>= 1  # bitwise equivalent of dividing by 2
    
    tested_bases = set()
    for _ in range(k):  # k times executing the composite test for n to be very certain of the primality of n
        a = random.randint(2, n-1)
        while a in tested_bases:
            a = random.randint(2, n-1)
        tested_bases.add(a)
        if pow(a, d, n) in {1, n-1}:
            continue
        continue_loop = False
        for i in range(s):
            if pow(a, 2**i * d, n) == n-1:
                continue_loop = True
                break
        if continue_loop:
            continue
        return False
    return True

In [20]:
num = generate(1024, isprime_miller_rabin)
num

118801047276814008917532747795788694751984585099894224914230698800690971616459669458310503112227271617416992683334030701544371244809762414382129407129635980505434333115184880005917599887245394357147641125164243705040540858460101123368959337671654629599128533465835931898178057909651645502601100638439067848559

In [21]:
!openssl prime {num}

A92DA586BF145ABD7338C43E8C241FD5A3D69E645262D39029085F1E76A85103B75D74419F48D2F50C8DEB0D4A52126126544A78CA19BDA3A5497047E29C8E9CCB781DF888D8A9AA3EE215EEF14B0186FCE76932299F86A338128D204AF89830778E3DD61FA5CEEB63D7868BD637AE0CE8F0CF35CDB8242F2D6AC59EEFE78F6F (118801047276814008917532747795788694751984585099894224914230698800690971616459669458310503112227271617416992683334030701544371244809762414382129407129635980505434333115184880005917599887245394357147641125164243705040540858460101123368959337671654629599128533465835931898178057909651645502601100638439067848559) is prime



<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercice 11.5: RSA cryptosystem

RSA is named after Ronald Rivest, Adi Shamir and Leonard Adleman who invented the algorithm in 1983. It is one of the most used asymmetric cryptosystems today. An asymmetric cryptosystem allows a person A to transmit an encrypted message to another person B without A having to know the secret keyword of B who would then use it to decrypt the message. The message from A generated with the public key is enough for B to decrypt the message with his corresponding private key. There are three big steps in the RSA algorithm:

**Creating keys.**
The person B wishing to receive a secret message chooses two very big prime numbers $p$ and $q$ which he keeps secret. He then calculates $n=pq$ and its Euler's totient $\varphi(n)=(p-1)(q-1)$ which counts the number of integers between 1 and $n-1$ co-prime with $n$. Then he chooses a number $e$ co-prime with $\varphi(n)$. The public key is formed by the couple $(n,e)$. Finally, the person B calculates the inverse $d$ of $e$ in modulus $\varphi(n)$, *i.e.* $d$ that satisfies $ed = 1 \pmod{\varphi(n)}$. The private key of B is $(p,q,d)$.

**Encrypting the message.**
To encrypt the message, the person A first transforms it into a whole number $M < n$. The encrypted message is then

$$
C = M^e \pmod n \,.
$$

**Decrypting the message.**
The encrypted message $C$ is transmitted to B, who then calculates

$$
M = C^d \pmod n \,
$$

which returns the original message.

**Attention.**

Make sure that the primes $p$ and $q$ must really be random, or else it would be "quite easy" to find their values. The random numbers generated by the module `random` follow Mersenne Twister's algorithm. This algorithm isn't considered cryptographically safe in a way that a wide enough observation of a few thousands random numbers generated by this algorithm suffice to predict all of its future iterations. To generate random numbers that are cryptographically safe, we have to use the module [`secrets`](https://docs.python.org/3.6/library/secrets.html).




**a)**
Demonstrate mathematically that the encrypted message actually corresponds to the original message.

<!-- --- begin hint in exercise --- -->

**Hints:**
If $a = b \pmod{\varphi(n)}$ and $M$ co-prime with $n$, then $M^a = M^b \pmod n$.

<!-- --- end hint in exercise --- -->



> In fact, based on Euler's theorem, $M$ must also be co-prime with $n$. Indeed, if $C = M^e \pmod{n}$ then $C^d = (M^e)^d = M^{ed} \pmod{n}$. Since $ed = 1 \pmod{\varphi (n)}$, $M$ co-prime with $n$ we have finally $M^{ed}=M \pmod {n}$.


**b)**
Given $e$ and $\varphi(n)$ write a function `bezout(e, phi)` determining $d$ satisfying $ed = 1 \pmod{\varphi(n)}$.

<!-- --- begin hint in exercise --- -->

**Hints:**
Use Euclid's generalized algorithm which determines the GCD $g$ of two numbers $a$ and $b$ as well as two numbers $x$ and $y$ satisfying $ax+by=g$.

<!-- --- end hint in exercise --- -->



In [22]:
def bezout(e, phi):
    s, s_ = 0, 1
    t, t_ = 1, 0
    r, r_ = phi, e
    
    while r != 0:
        q = r_ // r
        r_, r = r, r_ - q*r
        s_, s = s, s_ - q*s
        t_, t = t, t_ - q*t
    return s_ % phi

In [23]:
e, phi = 1351, 643
(bezout(e, phi) * e) % phi

1


**c)**
Write a function `generate_keys(length)` generating the primes $p$ and $q$ so that $n$ has `length` bits, determining $\varphi(n)$, $e$ and $d$, and finally returning the public and private keys, respectively $(n,e)$ and $(p,q,d)$.


In [24]:
import secrets

def generate_keys(length):
    def random_prime(bits):
        min_value = ceil(2**(bits-0.5))
        prime = secrets.randbits(bits)
        while prime < min_value or not isprime_miller_rabin(prime):
            prime = secrets.randbits(bits)
        return prime
    
    bits_p = secrets.choice(range(length//2 - 10, length//2 + 10))  # randomly choose a range for p's bits
    bits_q = length - bits_p
    p = random_prime(bits_p)
    q = random_prime(bits_q)
    n = p*q
    e = 65537
    phi = (p-1)*(q-1)
    d = bezout(e, phi) if bezout(e, phi) > 0 else phi + bezout(e, phi)  # Should be a positive integer
    
    return (n, e), (p, q, d)

In [25]:
length = 1024
public, private = generate_keys(length)
print("Public key:", public)
print("Private key:", private)

Public key: (152621768285099734219424123592859463968608803235212897377578807988823890409812992578398302650715642887428031721628182257239015699739248038898520457713716161642431929760589995711249534370427217048795464795950004460955889099679654376464906673749765296151769011347189073148559757518699123194823954023576529912147, 65537)
Private key: (26548181861001653224468442346916883286769201573981739673438006067496545426740167026744241650885565368421037326339405861041354443650448607230597133600180217, 5748859529597232107200388274764736163181604701068228127337902378966815677134955193228598712487023500052977222746635311156930806954328264936237100620357291, 80520168152732172824065924551729692024026397007197783537958174237782852965648321274863050070206815516055230248668935558940693147903996829164521466437423880987018375605461044355565197142502973208722923305263907023730112403578897449294669200156300729498006454046010043093596476006770465123809466314581071592193)


In [26]:
n, e = public
p, q, d = private

In [27]:
p*q == n

True

In [28]:
(e*d) % ((p-1)*(q-1))

1

In [29]:
len(bin(n)) - 2  # bin(n) returns a str typed binary value of n which includes '0b' at the beginning, therefore must be excluded to know the exact number of bits of n

1024

**d)**
By choosing to encode each character on 8 bits, a string of length $\ell$ can be written as a list $(a_0,a_1,\dots a_\ell)$ for each $0 \leq a_i \leq 255$. This list can then be converted to an integer $k$ as following:

$$
k = \sum_{i=0}^\ell a_i 256^i
$$

Write a function `toint` and another function `tostr` to respectively convert a string to an integer and back.

In [30]:
def toint(string):
    encrypted_msg = 0
    for i, char in enumerate(string):
        encrypted_msg += ord(char)*256**i
    return encrypted_msg

code = toint("I am legend is such a great movie starring Will Smith. I love watching him.")
code

752543656598430791546212673016081644515029171583346043640677252474922246724244942692995773015219717088247296046260360694581531473085574139295866474835470801196647884988166726295625

In [31]:
def tostr(num):
    msg = ""
    while num // 256 >= 0:
        a = 0
        while (num - a) % 256 != 0:
            a += 1
        msg += chr(a)
        num //= 256
        if num == 0:
            break
    return msg

tostr(code)

'I am legend is such a great movie starring Will Smith. I love watching him.'


**e)**
Write a function to encrypt a text with a public key and another function to decrypt it with the corresponding private key. To achieve this, we have to make sure that the text is convertible to a number less than $n$, if not then we split it into blocks to encrypt separately.

In [32]:
def break_msg(msg, k):
    """ Break the string into k even parts (in terms of number of characters) """
    q = len(msg) // k
    r = len(msg) % k
    return [msg[i*q : (i+1)*q + 1] if i<r else msg[i*q : (i+1)*q] for i in range(k)]

msg = 'I am legend is such a great movie starring Will Smith. I love watching him.'
break_msg(msg, 5)

['I am legend is ',
 'such a great mo',
 'vie starring Wi',
 'll Smith. I lov',
 'e watching him.']

In [33]:
public = (102879710770752511444645833011165696651619809409491556295029570602622028588515675930724362634624896351441630621506341531211272436681854793154606165896207790041954416913355391419643914918797618398109799014010931855695176217558393594050253975370232155523911584660100880097160695246305759491396478855962313664689,
          65537)
private = (79563557840776827903565737400968911552647625307192266548999184353943670464685632026945084663720144251159361004827848299962554503752610900101658946145770817,
           1293050657395640084328044917968166196339015056822260360955483553779516366396198317701676056519022318239979406312719222733835422685370445179884973796802417, 44178764670663715604722029974415005277423994021870407080138199878993419756268926983511233923222736134071163016022292288522191131506438186730397810806368517930378109939801855401198146020012241708454526590940672114387451626365242287279016559120153407419391811060195436769803350806644845587338005187189045113857)

In [34]:
def encrypt_rsa(msg, public_key):
    n, e = public_key
    num_chunks = 1
    
    while True:
        M_chunks = break_msg(msg, num_chunks)
        for chunk in M_chunks:
            if toint(chunk) >= n:
                num_chunks += 1
                break
        else:
            return map(lambda m: pow(toint(m), e, n), M_chunks)  # return a map object which needs to be iterated to get all values. This saves memory.
        
encrypted_msg = encrypt_rsa(msg, public)

In [35]:
for chunk in encrypted_msg:
    print(chunk)

34838993476127272364835782202842636025594803334530969370981019061591190351968216710544942983374820778436023611592069327989900736436669234559594636200925105866472312350487848263862314718077316782087595561403154902407566783259733796029033910216172780724922906067841837763975529097182397727302561235642214085369


In [36]:
def decrypt_rsa(chunk, private_key):
    p, q, d = private_key
    n = p * q
    decrypted = pow(chunk, d, n)
    return tostr(decrypted)

In [37]:
decrypt_rsa(chunk, private)

'I am legend is such a great movie starring Will Smith. I love watching him.'


<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercise 11.6: <span style="color:red">!!!</span> Crack the RSA cryptosystem

Below is a public key

In [38]:
public = (73722206893746878039310298412768333517547506486427363913406174815240823284857,
          33921003048397584579835360477549223828723590186917811609938274008840181968499)
n, e = public

and a message encrypted with this public key:

In [39]:
M = [22973877239788873882837788687834740958145091979501565167824992597825600406974,
     48503379361942356829127901273483580639426474600801539865525830360302350602689,
     2224798454942012298637628855810886175704245737423608868190613249161861526055,
     4500720701216145302036625979462397411127541711596515042635302843142748047486,
     35445000935671280429079877747553363121645781096430386417428307485228904386825,
     48627712259501563035806415688912560481020577805784144969245386699539463833735,
     71868389092768589092947834441169271512227882469129366706758279960751502739157,
     26019603019482382085505727901092092122241438959147654068117475447783602572984,
     65039729472521706954510624828984329309712256390395416184704490683377874857302,
     11805320141319662342135552217286927868466795280631816945032387836622798495117]

**a)**
Decrypt the above message !

<!-- --- begin hint in exercise --- -->

**Hints:**
Choose an appropriate algorithm, for example a quadratic sieving method (QS, MPQS, SIQS).

<!-- --- end hint in exercise --- -->

<!-- --- end exercise --- -->

In [40]:
p = 226521246220608157332267361877024940649
q = 325453828829588499457943240171704520593
phi = (p-1) * (q-1)
d = bezout(e, phi)

In [41]:
for chunk in M:
    print(decrypt_rsa(chunk, (p, q, d)))

Bonjour,
Félicitations, vous ven
ez de casser une clé RSA de 256 
bits !
Si vous avez coder vous-m
ême l'algorithme permettant de c
asser cette clef, vous avez gagn
é une plaque de chocolat que vou
s pouvez venir chercher dans mon
 bureau (Jussieu 16-26-301).
Ave
c mes meilleures salutations,
Ju
lien Guillod


> I admit that I cheated in finding the factorization of the public key's $n$ value because I cannot make my little iMac to crack this number using my implementation of the quadratic sieve.